In [1]:
import pandas as pd
import json

In [13]:
df = pd.read_json("data.json", encoding="utf-8")
print("Dữ liệu sau khi đọc với utf-8:")
print(df["address"])
df.to_csv('data.csv')
# Lưu lại DataFrame nếu cần (tùy chọn)
# df.to_json("data_fixed.json", orient="records", force_ascii=False)

Dữ liệu sau khi đọc với utf-8:
0       VietinBank - PGD CÃ ng Long, Sá»‘ 242 ÄÆ°á»n...
1       ATM VietinBank - CÃ´ng Ty TNHH Kinh Doanh Cháº...
2       ATM VietinBank - CÃ´ng Ty TNHH Kinh Doanh Cháº...
3       VietinBank - PGD PhÆ°á»›c ÄÃ´ng, áº¤p PhÆ°á»›...
4       VietinBank - PGD Tiá»ƒu Cáº§n, ÄÆ°á»ng trÃ¡n...
                              ...                        
3792    ATM VietinBank - TTTM Sense City CÃ  Mau, Sá»‘...
3793    VietinBank - PGD SÃ´ng Äá»‘c, KhÃ³m 10, Thá»‹...
3794    ATM VietinBank - Chá»£ NhÃ  Pháº¥n, XÃ£ Tháº¡n...
3795    VietinBank - Chi nhÃ¡nh CÃ  Mau, Sá»‘ 94 ÄÆ°á...
3796    VietinBank - PGD TÄƒÌc VÃ¢n, Sá»‘ 789 ÄÆ°á»...
Name: address, Length: 3797, dtype: object


In [16]:
import pandas as pd
import json

# Đọc dữ liệu từ file JSON (hoặc dùng dữ liệu của bạn)
# Giả sử bạn có file "data.json", thay bằng đường dẫn thực tế
df = pd.read_json("data.json", encoding="utf-8")

# Hàm sửa lỗi mã hóa cho cột address
def fix_encoding(text):
    if pd.notna(text):
        try:
            return text.encode('utf-8').decode('latin1')  # Đảo ngược mã hóa sai
        except UnicodeEncodeError:
            return text
    return text

# Áp dụng hàm sửa lỗi mã hóa cho cột address
df["address"] = df["address"].apply(fix_encoding)

# Hàm xử lý cột location thành point (latitude, longitude)
def extract_coordinates(location):
    # Nếu location là dictionary (đã được parse từ JSON)
    if isinstance(location, dict):
        coords = location['coordinates']
    # Nếu location là chuỗi, parse bằng json.loads
    elif isinstance(location, str):
        loc_dict = json.loads(location.replace("'", "\""))  # Thay ' bằng " để đúng cú pháp JSON
        coords = loc_dict['coordinates']
    else:
        raise ValueError(f"Không thể xử lý kiểu dữ liệu của location: {type(location)}")
    
    # Đổi thứ tự: [longitude, latitude] -> "latitude, longitude"
    return f"{coords[1]}, {coords[0]}"

# Tạo cột point từ cột location
df["point"] = df["location"].apply(extract_coordinates)

# Xóa cột location gốc nếu không cần nữa
df = df.drop(columns=["location"])

# Sắp xếp lại thứ tự cột
columns_order = ["point", "address", "name", "branch_manage", "type", "id", "atm_status", "title", "phone", "country", "gender", "leader", "morning_open", "morning_close", "afternoon_open", "afternoon_close"]
df = df[columns_order]

# Hiển thị vài dòng đầu để kiểm tra
print("Dữ liệu sau khi xử lý:")
print(df.head())

# Xuất ra file CSV
df.to_csv("data_fixed.csv", index=False, encoding="utf-8-sig")
print("Đã xuất dữ liệu ra file 'data_fixed.csv'")

Dữ liệu sau khi xử lý:
                      point  \
0        9.99973, 106.20532   
1      9.175483, 105.163118   
2      9.175483, 105.163118   
3  11.12485722, 106.3220518   
4        9.81405, 106.19212   

                                             address    name branch_manage  \
0  VietinBank - PGD CÃÂ ng Long, SÃ¡Â»â 242 Ã...  ATM-QR           980   
1  ATM VietinBank - CÃÂ´ng Ty TNHH Kinh Doanh Ch...  ATM-QR           860   
2  ATM VietinBank - CÃÂ´ng Ty TNHH Kinh Doanh Ch...  ATM-QR           860   
3  VietinBank - PGD PhÃÂ°Ã¡Â»âºc ÃÂÃÂ´ng, Ã¡...   R-ATM           664   
4  VietinBank - PGD TiÃ¡Â»Æu CÃ¡ÂºÂ§n, ÃÂÃÂ°Ã...  ATM-QR           980   

   type                                    id  atm_status title         phone  \
0     4  729a561e-dd03-4ff6-8a6f-7f6252ca3696           1  None  1900 558 868   
1     4  f6426eff-a9fe-4af5-b1db-4360b9ccfddf           1  None  1900 558 868   
2     4  3673dff7-1c43-405f-aafe-bdc768d543f1           1  None  1900 558 868 

In [ ]:
import pandas as pd

def fix_encoding_in_dataframe(df, columns_to_fix):
    """
    Đọc DataFrame pandas và cố gắng sửa lỗi encoding tiếng Việt trong các cột được chỉ định.

    Args:
        df (pd.DataFrame): DataFrame cần xử lý.
        columns_to_fix (list): Danh sách tên các cột cần sửa encoding.

    Returns:
        pd.DataFrame: DataFrame đã được sửa lỗi encoding (nếu có thể).
    """
    for col in columns_to_fix:
        if col in df.columns:
            try:
                # Thử giải mã bằng một số encoding phổ biến và mã hóa lại thành utf-8
                df[col] = df[col].str.encode('latin-1').str.decode('utf-8', errors='ignore')
            except UnicodeEncodeError:
                try:
                    df[col] = df[col].str.encode('windows-1258').str.decode('utf-8', errors='ignore')
                except UnicodeEncodeError:
                    print(f"Không thể tự động sửa encoding cho cột '{col}'.")
    return df

# Đường dẫn đến file CSV của bạn
csv_file_path = 'data.csv'

# Đọc file CSV
try:
    df = pd.read_csv(csv_file_path, encoding='utf-8') # Thử đọc với utf-8 trước
except UnicodeDecodeError:
    try:
        df = pd.read_csv(csv_file_path, encoding='latin-1') # Nếu utf-8 lỗi, thử latin-1
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(csv_file_path, encoding='windows-1258') # Thử windows-1258
        except UnicodeDecodeError:
            print("Lỗi: Không thể đọc file CSV với các encoding phổ biến.")
            exit()

# Xác định các cột chứa địa chỉ hoặc text tiếng Việt bị lỗi encoding
columns_to_fix = ['address', 'name'] # Thêm các cột khác nếu cần

# Sửa lỗi encoding trong các cột được chỉ định
df_fixed = fix_encoding_in_dataframe(df.copy(), columns_to_fix)

# In ra DataFrame đã sửa (hoặc lưu vào file mới)
print("DataFrame sau khi sửa lỗi encoding:")
display(df_fixed)

# Nếu bạn muốn lưu DataFrame đã sửa vào một file CSV mới:
df_fixed.to_csv('your_fixed_data.csv', index=False, encoding='utf-8')

DataFrame sau khi sửa lỗi encoding:


,Unnamed: 0,location,address,name,branch_manage,type,id,atm_status,title,phone,country,gender,leader,morning_open,morning_close,afternoon_open,afternoon_close
0,0,"{'type': 'Point', 'coordinates': [106.20532, 9...","VietinBank - PGD Càng Long, Số 242 ?ư?ng Quốc ...",ATM-QR,980,4,729a561e-dd03-4ff6-8a6f-7f6252ca3696,1,NaN,1900 558 868,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"{'type': 'Point', 'coordinates': [105.163118, ...",ATM VietinBank - Công Ty TNHH Kinh Doanh Chế B...,ATM-QR,860,4,f6426eff-a9fe-4af5-b1db-4360b9ccfddf,1,NaN,1900 558 868,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"{'type': 'Point', 'coordinates': [105.163118, ...",ATM VietinBank - Công Ty TNHH Kinh Doanh Chế B...,ATM-QR,860,4,3673dff7-1c43-405f-aafe-bdc768d543f1,1,NaN,1900 558 868,0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"{'type': 'Point', 'coordinates': [106.3220518,...","VietinBank - PGD Phước ?ông, Ấp Phước ?ức, Xã ...",R-ATM,664,3,7e3590a8-e4de-4cea-97a2-de2052ef3d59,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"{'type': 'Point', 'coordinates': [106.19212, 9...","VietinBank - PGD Tiểu Cần, ?ư?ng tránh Quốc lộ...",ATM-QR,980,4,8b33a8d8-8828-491a-a3c8-942ff2eaa2a3,1,NaN,1900 558 868,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3792,3792,"{'type': 'Point', 'coordinates': [105.154446, ...","ATM VietinBank - TTTM Sense City Cà Mau, Số 09...",ATM-QR,860,4,7ce99d1b-ab54-4645-bc65-1f1c4f2dcc8c,1,NaN,1900 558 868,0,NaN,NaN,NaN,NaN,NaN,NaN
3793,3793,"{'type': 'Point', 'coordinates': [104.83707, 9...","VietinBank - PGD Sông ?ốc, Khóm 10, Thị trấn S...",ATM-QR,860,4,4b59618d-7e05-4616-9f89-6e0597818c34,1,NaN,1900 558 868,0,NaN,NaN,NaN,NaN,NaN,NaN
3794,3794,"{'type': 'Point', 'coordinates': [105.099933, ...","ATM VietinBank - Chợ Nhà Phấn, Xã Thạnh Phú, H...",ATM-QR,860,4,303b8975-435b-4d1d-a4be-ca05c1fa53ec,1,NaN,1900 558 868,0,NaN,NaN,NaN,NaN,NaN,NaN
3795,3795,"{'type': 'Point', 'coordinates': [105.153592, ...","VietinBank - Chi nhánh Cà Mau, Số 94 ?ư?ng Lý ...",ATM-QR,860,4,38c1b45f-eb58-48fe-a2d7-80567adc4e65,1,NaN,1900 558 868,0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd

def fix_encoding_in_dataframe(df, columns_to_fix):
    """
    Đọc DataFrame pandas và cố gắng sửa lỗi encoding tiếng Việt trong các cột được chỉ định.

    Args:
        df (pd.DataFrame): DataFrame cần xử lý.
        columns_to_fix (list): Danh sách tên các cột cần sửa encoding.

    Returns:
        pd.DataFrame: DataFrame đã được sửa lỗi encoding (nếu có thể).
    """
    for col in columns_to_fix:
        if col in df.columns:
            # Hàm sửa từng chuỗi
            def fix_single_string(text):
                if pd.isna(text):
                    return text
                # Thử phương pháp mã hóa ngược trước (utf-8 -> latin-1)
                try:
                    return text.encode('utf-8').decode('latin-1')
                except (UnicodeEncodeError, UnicodeDecodeError):
                    # Nếu lỗi, thử encode latin-1 -> decode utf-8
                    try:
                        return text.encode('latin-1').decode('utf-8', errors='ignore')
                    except (UnicodeEncodeError, UnicodeDecodeError):
                        # Nếu vẫn lỗi, thử encode windows-1258 -> decode utf-8
                        try:
                            return text.encode('windows-1258').decode('utf-8', errors='ignore')
                        except (UnicodeEncodeError, UnicodeDecodeError):
                            print(f"Không thể tự động sửa encoding cho một giá trị trong cột '{col}'.")
                            return text

            # Áp dụng hàm sửa cho cột
            df[col] = df[col].apply(fix_single_string)

            # Thay thế thủ công các ký tự sai thường gặp
            replacements = {
                '?ư?ng': 'Đường',
                '?ông': 'Đông',
                '?ức': 'Đức',
                'Ph?ng': 'Phường',
                'S?': 'Số',
                'Tr?n': 'Trần',
                'Nguy?n': 'Nguyễn',
                'H?': 'Hồ',
                'Qu?c': 'Quốc',
                'Th?': 'Thị'
            }
            for wrong, correct in replacements.items():
                df[col] = df[col].str.replace(wrong, correct, regex=False)

    return df

# Đường dẫn đến file CSV của bạn
csv_file_path = 'data.csv'

# Đọc file CSV
try:
    df = pd.read_csv(csv_file_path, encoding='utf-8')  # Thử đọc với utf-8 trước
except UnicodeDecodeError:
    try:
        df = pd.read_csv(csv_file_path, encoding='latin-1')  # Nếu utf-8 lỗi, thử latin-1
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(csv_file_path, encoding='windows-1258')  # Thử windows-1258
        except UnicodeDecodeError:
            print("Lỗi: Không thể đọc file CSV với các encoding phổ biến.")
            exit()

# Xác định các cột chứa địa chỉ hoặc text tiếng Việt bị lỗi encoding
columns_to_fix = ['address', 'name']  # Thêm các cột khác nếu cần

# Sửa lỗi encoding trong các cột được chỉ định
df_fixed = fix_encoding_in_dataframe(df.copy(), columns_to_fix)

# In ra DataFrame đã sửa
print("DataFrame sau khi sửa lỗi encoding:")
print(df_fixed.head())

# Nếu bạn muốn lưu DataFrame đã sửa vào một file CSV mới:
df_fixed.to_csv('your_fixed_data.csv', index=False, encoding='utf-8-sig')  # Sử dụng utf-8-sig để hỗ trợ tiếng Việt trong Excel
print("Đã lưu DataFrame đã sửa vào 'your_fixed_data_2.csv'")

DataFrame sau khi sửa lỗi encoding:
   Unnamed: 0                                           location  \
0           0  {'type': 'Point', 'coordinates': [106.20532, 9...   
1           1  {'type': 'Point', 'coordinates': [105.163118, ...   
2           2  {'type': 'Point', 'coordinates': [105.163118, ...   
3           3  {'type': 'Point', 'coordinates': [106.3220518,...   
4           4  {'type': 'Point', 'coordinates': [106.19212, 9...   

                                             address    name branch_manage  \
0  VietinBank - PGD CÃÂ ng Long, SÃ¡Â»Â 242 Ã?...  ATM-QR           980   
1  ATM VietinBank - CÃÂ´ng Ty TNHH Kinh Doanh Ch...  ATM-QR           860   
2  ATM VietinBank - CÃÂ´ng Ty TNHH Kinh Doanh Ch...  ATM-QR           860   
3  VietinBank - PGD PhÃÂ°Ã¡Â»Âc Ã?ÃÂ´ng, Ã¡Âº...   R-ATM           664   
4  VietinBank - PGD TiÃ¡Â»Âu CÃ¡ÂºÂ§n, Ã?ÃÂ°Ã¡...  ATM-QR           980   

   type                                    id  atm_status title         phone  \
0    

: 